# installing packages

In [1]:
!pip install cmake
!pip install ffmpeg-python
!pip install dlib
!pip install face_recognition
!pip install moviepy
from IPython.display import clear_output
clear_output()
print("\nDone")


Done


# trimming face and no face video and trimming audio

## 1st method for face detection (getting few frame in face video with no face)

In [ ]:
import cv2
import face_recognition
import numpy as np
import moviepy.editor as mp

# Load the video
video_path = "/content/input_video.mp4"  # Update with your video file path
audio_path = "/content/input_audio.wav"  # Update with your audio file path
video = cv2.VideoCapture(video_path)

# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)

# Create lists to store frames and timestamps
frames_with_faces = []
timestamps_with_faces = []
frames_without_faces = []
timestamps_without_faces = []

# # Enable CUDA for GPU acceleration
# cv2.cuda.setDevice(0)  # Set the GPU device number, if multiple GPUs are available

while True:
    ret, frame = video.read()

    if not ret:
        break

    # Convert the frame to RGB for face_recognition library
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # # Transfer frame data to GPU memory
    # gpu_frame = cv2.cuda_GpuMat()
    # gpu_frame.upload(rgb_frame)

    # Detect faces in the frame using GPU-accelerated face recognition
    face_locations = face_recognition.face_locations(rgb_frame)

    if len(face_locations) > 0:
        # Store frames with faces and corresponding timestamps
        frames_with_faces.append(frame)
        timestamps_with_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)
    else:
        # Store frames without faces and corresponding timestamps
        frames_without_faces.append(frame)
        timestamps_without_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)

# Create video writer for frames with faces
writer_with_faces = cv2.VideoWriter("/content/video_with_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_with_faces[0].shape[1], frames_with_faces[0].shape[0]))
for frame in frames_with_faces:
    writer_with_faces.write(frame)
writer_with_faces.release()

# Create video writer for frames without faces
writer_without_faces = cv2.VideoWriter("/content/video_without_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_without_faces[0].shape[1], frames_without_faces[0].shape[0]))
for frame in frames_without_faces:
    writer_without_faces.write(frame)
writer_without_faces.release()


# Save timestamps of frames with faces
np.savetxt("timestamps_with_faces.txt", timestamps_with_faces)

# Save timestamps of frames without faces
np.savetxt("timestamps_without_faces.txt", timestamps_without_faces)


# Release the video object
video.release()

In [8]:
import moviepy.editor as mp

audio_path = "/content/input_audio.wav"  # Update with your audio file path

# Trim audio based on timestamps of frames with faces
audio_with_faces = mp.AudioFileClip(audio_path).subclip(timestamps_with_faces[0], timestamps_with_faces[-1])
audio_with_faces.write_audiofile("audio_with_faces.mp3")

# Trim audio based on timestamps of frames without faces
audio_without_faces = mp.AudioFileClip(audio_path).subclip(timestamps_without_faces[0], timestamps_without_faces[-1])
audio_without_faces.write_audiofile("audio_without_faces.mp3")

MoviePy - Writing audio in audio_with_faces.mp3


MoviePy - Done.
MoviePy - Writing audio in audio_without_faces.mp3


MoviePy - Done.


## 2nd method for face detection working fine, (but still need some tuning)

In [8]:
import cv2
import face_recognition
import numpy as np
import moviepy.editor as mp

# Load the video
video_path = "/content/input_video.mp4"  # Update with your video file path
audio_path = "/content/input_audio.wav"  # Update with your audio file path
video = cv2.VideoCapture(video_path)

# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)

# Create lists to store frames and timestamps
frames_with_faces = []
timestamps_with_faces = []
frames_without_faces = []
timestamps_without_faces = []

# # Enable CUDA for GPU acceleration
# cv2.cuda.setDevice(0)  # Set the GPU device number, if multiple GPUs are available

# Load Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Parameters for face detection
scale_factor = 1.3  # Adjust the scale factor for fine-tuning face detection sensitivity
min_neighbors = 5  # Adjust the minimum number of neighbors for fine-tuning face detection accuracy
min_size = (30, 30)  # Adjust the minimum face size for fine-tuning face detection

while True:
    ret, frame = video.read()

    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # # Transfer frame data to GPU memory
    # gpu_frame = cv2.cuda_GpuMat()
    # gpu_frame.upload(rgb_frame)

    # # Detect faces in the frame using GPU-accelerated face recognition
    # face_locations = face_recognition.face_locations(gray_frame)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=scale_factor, minNeighbors=min_neighbors, minSize=min_size)

    if len(faces) > 0:
        # Store frames with faces and corresponding timestamps
        frames_with_faces.append(frame)
        timestamps_with_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)
    else:
        # Store frames without faces and corresponding timestamps
        frames_without_faces.append(frame)
        timestamps_without_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)

# Create video writer for frames with faces
writer_with_faces = cv2.VideoWriter("/content/video_with_faces_gray.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_with_faces[0].shape[1], frames_with_faces[0].shape[0]))
for frame in frames_with_faces:
    writer_with_faces.write(frame)
writer_with_faces.release()

# Create video writer for frames without faces
writer_without_faces = cv2.VideoWriter("/content/video_without_faces_gray.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_without_faces[0].shape[1], frames_without_faces[0].shape[0]))
for frame in frames_without_faces:
    writer_without_faces.write(frame)
writer_without_faces.release()

# Save timestamps of frames with faces
np.savetxt("timestamps_with_faces_gray.txt", timestamps_with_faces)

# Save timestamps of frames without faces
np.savetxt("timestamps_without_faces_gray.txt", timestamps_without_faces)

# Trim audio based on timestamps of frames with faces
audio_with_faces = mp.AudioFileClip(audio_path).subclip(timestamps_with_faces[0], timestamps_with_faces[-1])
audio_with_faces.write_audiofile("audio_with_faces_gray.mp3")

# Trim audio based on timestamps of frames without faces
audio_without_faces = mp.AudioFileClip(audio_path).subclip(timestamps_without_faces[0], timestamps_without_faces[-1])
audio_without_faces.write_audiofile("audio_without_faces_gray.mp3")

# Release the video object
video.release()

MoviePy - Writing audio in audio_with_faces_gray.mp3


MoviePy - Done.
MoviePy - Writing audio in audio_without_faces_gray.mp3


MoviePy - Done.


In [ ]:
#@title <h1>audio trimmer</h1>
import moviepy.editor as mp

audio_path = "/content/input_audio.wav"  # Update with your audio file path

# Trim audio based on timestamps of frames with faces
audio_with_faces = mp.AudioFileClip(audio_path).subclip(timestamps_with_faces[0], timestamps_with_faces[-1])
audio_with_faces.write_audiofile("audio_with_faces_gray.mp3")

# Trim audio based on timestamps of frames without faces
audio_without_faces = mp.AudioFileClip(audio_path).subclip(timestamps_without_faces[0], timestamps_without_faces[-1])
audio_without_faces.write_audiofile("audio_without_faces_gray.mp3")

# putting audio on face and no face library (optional)

In [9]:
# Set audio for video with faces
video_with_faces = mp.VideoFileClip("/content/video_with_faces.mp4")
video_with_faces = video_with_faces.set_audio(audio_with_faces)
video_with_faces.write_videofile("video_with_faces_audio.mp4", fps=fps, codec="libx264", audio_codec="aac")

# Set audio for video without faces
video_without_faces = mp.VideoFileClip("/content/video_without_faces.mp4")
video_without_faces = video_without_faces.set_audio(audio_without_faces)
video_without_faces.write_videofile("video_without_faces_audio.mp4", fps=fps, codec="libx264", audio_codec="aac")

Moviepy - Building video video_with_faces_audio.mp4.
MoviePy - Writing audio in video_with_faces_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_with_faces_audio.mp4



Moviepy - Done !
Moviepy - video ready video_with_faces_audio.mp4
Moviepy - Building video video_without_faces_audio.mp4.
MoviePy - Writing audio in video_without_faces_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_without_faces_audio.mp4



Moviepy - Done !
Moviepy - video ready video_without_faces_audio.mp4


# Preparing and applying model

In [10]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [11]:
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/video_with_faces_gray.mp4" --audio "/content/input_audio.wav"

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 1262
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 5386)
Length of mel chunks: 2016
  0% 0/16 [00:00<?, ?it/s]
  0% 0/79 [00:00<?, ?it/s]
  1% 1/79 [00:43<55:56, 43.03s/it]
  3% 2/79 [00:46<25:00, 19.49s/it]
  4% 3/79 [00:48<14:54, 11.77s/it]
  5% 4/79 [00:51<10:05,  8.08s/it]
  6% 5/79 [00:53<07:31,  6.10s/it]
  8% 6/79 [00:56<06:02,  4.96s/it]
  9% 7/79 [00:58<05:00,  4.17s/it]
 10% 8/79 [01:01<04:19,  3.65s/it]
 11% 9/79 [01:04<03:51,  3.31s/it]
 13% 10/79 [01:06<03:29,  3.04s/it]
 14% 11/79 [01:09<03:16,  2.89s/it]
 15% 12/79 [01:11<03:09,  2.83s/it]
 16% 13/79 [01:14<03:00,  2.73s/it]
 18% 14/79 [01:16<02:51,  2.64s/it]
 19% 15/79 [01:19<02:46,  2.60s/it]
 20% 16/79 [01:21<02:40,  2.55s/it]
 

In [12]:
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Rough work

In [ ]:
import cv2
import face_recognition

# # Mount Google Drive to access video file
# from google.colab import drive
# drive.mount('/content/drive')

# Load the video
video_path = "/content/input_video.mp4"  # Update with your video file path
video = cv2.VideoCapture(video_path)

# Create lists to store frames
frames_with_faces = []
frames_without_faces = []

# # Enable CUDA for GPU acceleration
# cv2.cuda.setDevice(0)  # Set the GPU device number, if multiple GPUs are available

while True:
    ret, frame = video.read()

    if not ret:
        break

    # Convert the frame to RGB for face_recognition library
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(rgb_frame)

    if len(face_locations) > 0:
        # Store frames with faces
        frames_with_faces.append(frame)
    else:
        # Store frames without faces
        frames_without_faces.append(frame)

# Create video writer for frames with faces
writer_with_faces = cv2.VideoWriter("/content/video_with_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_with_faces[0].shape[1], frames_with_faces[0].shape[0]))
for frame in frames_with_faces:
    writer_with_faces.write(frame)
writer_with_faces.release()

# Create video writer for frames without faces
writer_without_faces = cv2.VideoWriter("/content/video_without_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_without_faces[0].shape[1], frames_without_faces[0].shape[0]))
for frame in frames_without_faces:
    writer_without_faces.write(frame)
writer_without_faces.release()

# Release the video object
video.release()

with timestamp

In [ ]:
import cv2
import face_recognition
import numpy as np

# Load the video
video_path = "/content/input_video.mp4"  # Update with your video file path
video = cv2.VideoCapture(video_path)

# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)

# Create lists to store frames and timestamps
frames_with_faces = []
timestamps_with_faces = []
frames_without_faces = []
timestamps_without_faces = []

# # Enable CUDA for GPU acceleration
# cv2.cuda.setDevice(0)  # Set the GPU device number, if multiple GPUs are available

while True:
    ret, frame = video.read()

    if not ret:
        break

    # Convert the frame to RGB for face_recognition library
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # # Transfer frame data to GPU memory
    # gpu_frame = cv2.cuda_GpuMat()
    # gpu_frame.upload(rgb_frame)

    # Detect faces in the frame using GPU-accelerated face recognition
    face_locations = face_recognition.face_locations(rgb_frame)

    if len(face_locations) > 0:
        # Store frames with faces and corresponding timestamps
        frames_with_faces.append(frame)
        timestamps_with_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)
    else:
        # Store frames without faces and corresponding timestamps
        frames_without_faces.append(frame)
        timestamps_without_faces.append(video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)

# Create video writer for frames with faces
writer_with_faces = cv2.VideoWriter("/content/video_with_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_with_faces[0].shape[1], frames_with_faces[0].shape[0]))
for frame in frames_with_faces:
    writer_with_faces.write(frame)
writer_with_faces.release()

# Create video writer for frames without faces
writer_without_faces = cv2.VideoWriter("/content/video_without_faces.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames_without_faces[0].shape[1], frames_without_faces[0].shape[0]))
for frame in frames_without_faces:
    writer_without_faces.write(frame)
writer_without_faces.release()

# Save timestamps of frames with faces
np.savetxt("timestamps_with_faces.txt", timestamps_with_faces)

# Save timestamps of frames without faces
np.savetxt("timestamps_without_faces.txt", timestamps_without_faces)

# Release the video object
video.release()

# 2nd (txt to speech) NOT ATTEMPTING

In [ ]:
# Install dependencies
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install inflect

In [ ]:
!pip install librosa

In [ ]:
!pip install Unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install Pillow

In [ ]:
# Import necessary libraries
import tensorflow as tf

In [ ]:
# Clone the Tacotron 2 repository
!git clone https://github.com/NVIDIA/tacotron2.git
%cd tacotron2

Cloning into 'tacotron2'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 412 (delta 2), reused 3 (delta 0), pack-reused 406
Receiving objects: 100% (412/412), 2.70 MiB | 3.13 MiB/s, done.
Resolving deltas: 100% (202/202), done.
/content/tacotron2


In [ ]:
# Download pre-trained model checkpoints (e.g., Tacotron 2 and WaveGlow)
!wget <link_to_tacotron2_checkpoint>
!wget <link_to_waveglow_checkpoint>

In [ ]:
# Run the inference script
!python inference.py --tacotron2 <path_to_tacotron2_checkpoint> --waveglow <path_to_waveglow_checkpoint> --text "Hello, how are you?"